# 第7回：畳み込みニューラルネットワーク (3)

## 到達目標
* 畳み込みニューラルネットワーク (CNN) が開発されるまでの歴史を概観する
* CNNを用いて画像分類タスクを実装・実行できる

## キーワード
* 単純型・複雑型細胞
* 畳み込み層
* プーリング層
* 全結合層
* 特徴マップ
* AlexNet

### 実践演習 (85分)

#### 課題1: Chainerを用いたCNNの実装 (50分)
* MNISTデータセットを用いて、CNNの学習に挑戦
* 第6回までの知識と合わせて、正解率99%を目指そう
* 使えるツール：畳み込み層、プーリング層、Batch Normalization、SGD、Adam、Dropout

In [1]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')

In [2]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

In [3]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [4]:
#Number of images in each mini-batch
batchsize=100

#Number of sweeps over the dataset to train
epoch=20

#Frequency of taking a snapshot
frequency=-1

#GPU ID (negative value indicates CPU)
gpu=-1

#Directory to output the result
out='result'

#Resume the training from snapshot
resume=''

#Number of units
unit=1000

In [5]:
# Load the MNIST dataset
train, test = chainer.datasets.get_mnist()

In [6]:
model = L.Classifier(MLP(unit, 10))

In [7]:
# Setup an optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [8]:
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize,
                                             repeat=False, shuffle=False)

In [9]:
# Set up a trainer
updater = training.StandardUpdater(train_iter, optimizer, device=gpu)
trainer = training.Trainer(updater, (epoch, 'epoch'), out=out)

# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
frequency = epoch if frequency == -1 else max(1, frequency)
trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

In [10]:
# Save two plot images to the result dir
if extensions.PlotReport.available():
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                              'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'],
                              'epoch', file_name='accuracy.png'))

In [11]:
# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

In [12]:
if resume:
    # Resume from a snapshot
    chainer.serializers.load_npz(resume, trainer)

In [13]:
# Run the training
trainer.run()

     total [..................................................]  0.83%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    25.905 iters/sec. Estimated time to finish: 0:07:35.518108.
     total [#.................................................]  2.50%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 20 epochs
    25.732 iters/sec. Estimated time to finish: 0:07:34.686859.
     total [#.................................................]  3.33%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 20 epochs
    26.141 iters/sec. Estimated time to finish: 0:07:23.742375.
     total [##.....................

     total [##############....................................] 28.33%
this epoch [#################################.................] 66.67%
      3400 iter, 5 epoch / 20 epochs
    22.307 iters/sec. Estimated time to finish: 0:06:25.532597.
     total [##############....................................] 29.17%
this epoch [#########################################.........] 83.33%
      3500 iter, 5 epoch / 20 epochs
    22.398 iters/sec. Estimated time to finish: 0:06:19.504940.
     total [###############...................................] 30.00%
this epoch [..................................................]  0.00%
      3600 iter, 6 epoch / 20 epochs
    22.212 iters/sec. Estimated time to finish: 0:06:18.166543.
     total [###############...................................] 30.83%
this epoch [########..........................................] 16.67%
      3700 iter, 6 epoch / 20 epochs
    22.308 iters/sec. Estimated time to finish: 0:06:12.061239.
     total [###############.

     total [###########################.......................] 55.83%
this epoch [########..........................................] 16.67%
      6700 iter, 11 epoch / 20 epochs
    22.264 iters/sec. Estimated time to finish: 0:03:58.051814.
     total [############################......................] 56.67%
this epoch [################..................................] 33.33%
      6800 iter, 11 epoch / 20 epochs
    22.291 iters/sec. Estimated time to finish: 0:03:53.278748.
     total [############################......................] 57.50%
this epoch [#########################.........................] 50.00%
      6900 iter, 11 epoch / 20 epochs
    22.314 iters/sec. Estimated time to finish: 0:03:48.552403.
     total [#############################.....................] 58.33%
this epoch [#################################.................] 66.67%
      7000 iter, 11 epoch / 20 epochs
    22.354 iters/sec. Estimated time to finish: 0:03:43.669556.
     total [############

     total [#########################################.........] 83.33%
this epoch [#################################.................] 66.67%
     10000 iter, 16 epoch / 20 epochs
    21.615 iters/sec. Estimated time to finish: 0:01:32.527535.
     total [##########################################........] 84.17%
this epoch [#########################################.........] 83.33%
     10100 iter, 16 epoch / 20 epochs
    21.614 iters/sec. Estimated time to finish: 0:01:27.907115.
     total [##########################################........] 85.00%
this epoch [..................................................]  0.00%
     10200 iter, 17 epoch / 20 epochs
    21.501 iters/sec. Estimated time to finish: 0:01:23.717022.
     total [##########################################........] 85.83%
this epoch [########..........................................] 16.67%
     10300 iter, 17 epoch / 20 epochs
    21.454 iters/sec. Estimated time to finish: 0:01:19.239348.
     total [############

#### 課題2a：(パイロット用) Data Augmentation (20分)
* Horizontal flippingの実装 (行列のスライス)
* Scale augmentation (scipy.misc.imresize)
* Random crop/padding (np.pad)

#### 課題2b：(本番用) 未定 (20分)

#### 解説・コードレビュー (15分)

### フィードバック・次回予告 (5分)

## 参考文献 (講師の方も随時追加お願いします)